# Producing Project Plans
This notebook is for experimenting with having an LLM plan out a coding project.

In [ ]:
from dotenv import load_dotenv
import os
import warnings
import subprocess
from openai import OpenAI
from pathlib import Path

warnings.filterwarnings("ignore")
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("OPENAI_API_KEY is set correctly.")
else:
    print("OPENAI_API_KEY is not set.")

In [2]:
architecture_policies = [
    {
        "name": "Modularity and Function-Oriented Design",
        "description": "Design the codebase as a collection of small, focused functions, each performing one clear task.",
        "policies": [
            "Each function should have a single responsibility.",
            "Functions should be stateless where possible.",
            "Avoid side effects (e.g., global state modifications).",
            "Write reusable and composable functions.",
        ],
    },
    {
        "name": "CSV as the Core Data Format",
        "description": "Treat CSV files as the primary medium for data input, output, and inter-process communication.",
        "policies": [
            "Input and output data should default to CSV files instead of strings.",
            "Functions should focus on reading from and writing to CSV files.",
            "Encapsulate CSV parsing and writing logic into dedicated utility functions.",
            "Ensure CSV schemas are consistent and well-documented.",
        ],
    },
    {
        "name": "Separation of Concerns",
        "description": "Keep concerns (e.g., data handling, computation, and presentation) isolated.",
        "policies": [
            "Separate data processing, business logic, and I/O operations.",
            "Avoid mixing CSV parsing with computation logic in the same function.",
            "Use interfaces or adapters for external dependencies (e.g., file systems).",
        ],
    },
    {
        "name": "Data Flow Transparency",
        "description": "Ensure clarity in how data moves through the system.",
        "policies": [
            "Functions should explicitly accept inputs and return outputs.",
            "Avoid relying on hidden state or global variables.",
            "Use clear pipelines when chaining multiple data transformations.",
        ],
    },
    {
        "name": "Minimalism (Unix Philosophy)",
        "description": "Build small, sharp tools that can be combined in pipelines.",
        "policies": [
            "Functions should do one thing well.",
            "Allow for chaining outputs to other scripts/tools.",
            "Favor simplicity over cleverness in implementation.",
            "Document command-line or API interfaces for integration.",
        ],
    },
]
policy_list = list(
    set(policy for item in architecture_policies for policy in item.get("policies", []))
)
arch_policies = "\n\t- ".join(policy_list)

In [3]:
plan_format_policies = [
    {
        "name": "MD040",
        "description": "Fenced code blocks should have a language specified",
        "policies": [
            "Fenced code blocks should have a language identifier.",
            "Use the appropriate language identifier for the code block.",
            "Ensure the language identifier is supported by the markdown parser.",
        ],
    },
    {
        "name": "Avoid non-plan content",
        "description": "Ensure that the response is restricted to the plan, without additional commentary.",
        "policies": [
            "Avoid adding non-plan content to the response.",
            "Keep the response relevant to the task at hand.",
        ],
    },
]
policy_list = list(
    set(policy for item in plan_format_policies for policy in item.get("policies", []))
)
format_policies = "\n\t- ".join(policy_list)

In [4]:
project_plan_template = """
#       PROJECT PLAN  TEMPLATE  #

## 1. Architectural Style
<choose_one>
- Layered Architecture: Presentation, Business Logic, Data Access, Database
- Microservices Architecture: Independent services for specific capabilities
- Modular Monolith: Independent modules, shared deployment
- Event-Driven Architecture: Event producers, consumers, queues

## 2. Component Interface Definition
- Accepted Data: [Data input]
- Returned Data: [Data Output]

## 3. Domain-Driven Design (DDD)
Define bounded contexts:
- User Context: [Responsibilities]
- Order Context: [Responsibilities]
- Payment Context: [Responsibilities]

## 4. Dependency Management
- Dependencies Between Components: [Dependencies]
- Cyclic Dependency Avoidance: [Strategies]
- Dependency Injection: [Yes/No, describe]

## 5. Data Flow Between Sub-Components
Describe data flow:
- Sequence Diagram: [Interactions as ASCII art]
- Data Flow Diagram: [Flow as ASCII art]
- Component Diagram: [Dependencies between components as ASCII art]

## 6. Component Documentation
Document each component:
- **Name:** [Unique Identifier]
- **Purpose:** [Description of purpose]
- **Inputs:** [Expected inputs]
- **Outputs:** [Expected outputs]
- **Dependencies:** [List dependencies]

#       END OF TEMPLATE      #
"""

In [5]:
project_ideas = {
    "summarizer": "Summarizer for long text documents, generating concise summaries.",
    "email_analyzer": "Email analyzer to find key participants, topics, and action items.",
    "task_prioritizer": "Task prioritizer using CSV input for deadlines, importance, and dependencies.",
    "data_cleaner": "Data cleaner for CSVs, detecting missing values and standardizing formats.",
    "web_scraper": "Web scraper to extract key info from URLs and save it into a CSV file.",
}

In [6]:
project_plan_prompt_preamble = """ 
You are a software engineer tasked with designing a new project.
Your goal is to create a project plan that follows best practices in software architecture and coding.
"""

In [7]:
def format_plan(project_name, plan):
    "Removes template header/footer and adds title."
    start = plan.find("## 1.")
    end = plan.rfind("END OF TEMPLATE")
    plan = plan[start:end].strip().strip("#").strip()
    return f"# {project_name}\n{plan}\n"

In [8]:
client = OpenAI(api_key=openai_api_key)

for project_name, project_idea in project_ideas.items():
    project_plan_prompt = f"""{project_plan_prompt_preamble}
    Your project should adhere to the following policies::\n {arch_policies}\n
    Solve this problem:\n {project_idea}\n
    Follow this template (removing the template header/footer):\n {project_plan_template}\n
    Format the plan according to these policies:\n {format_policies}\n
    """
    response = client.chat.completions.create(
        model="o1-mini", messages=[{"role": "user", "content": project_plan_prompt}]
    )
    plan = format_plan(project_name, response.choices[0].message.content)

    # Create directory and write file
    projects_path = Path("projects")
    projects_path.mkdir(exist_ok=True)
    project_path = projects_path / project_name
    project_path.mkdir(exist_ok=True)
    (project_path / "plan.md").write_text(plan)

    # format the plan to be good markdown
    subprocess.run(["mdformat", str(project_path / "plan.md")])